In [8]:
from Explanations_Models.Custom_DT.LIME import LIME
from FeatureImportance.FI import FeatureImportance
import yaml
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
import json
import warnings
from tqdm import tqdm
from EnvRunner import GymRunner


# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
# parser = argparse.ArgumentParser()

# parser.add_argument("--config_filename", required=False, type=str)
domain = "cartpole"
config_file = open("config_envs/{}.yml".format(domain))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format("cartpole"), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)
model = Runner.model.policy.network

device:  cuda


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possibl

In [41]:
def get_gradient(model, point):
    """
    Computes the gradient of the model output with respect to the input point.
    
    Args:
        model: a PyTorch model
        point: a 1D torch tensor of shape (input_dim,) or (1, input_dim)

    Returns:
        gradient: a 1D torch tensor of shape (input_dim,)
    """
    point = torch.tensor(point, dtype=torch.float32)
    model.eval()
    point = point.clone().detach().requires_grad_(True)

    output = model(point)
    # if output.ndim > 1:
    #     output = output.squeeze()

    # If scalar output
    #assert output.numel() == 1, "Output must be scalar for gradient calculation"
    output.backward()

    gradient = point.grad.detach().clone()
    return gradient

def smooth_grad(model, input_tensor, n_samples=50, noise_std=2., multiply_by_input=False):
    """
    Computes SmoothGrad explanation by averaging noisy gradients.
    
    Args:
        model: PyTorch model (should be in eval mode)
        input_tensor: 1D torch.Tensor of shape [input_dim]
        n_samples: number of noisy samples to draw
        noise_std: standard deviation of Gaussian noise to add
        multiply_by_input: if True, returns gradient * input (Gradient × Input)

    Returns:
        smooth_grad: torch.Tensor of shape [input_dim]
    """
    model.eval()
    grads = []

    for _ in range(n_samples):
        # Add noise to input
        noisy_input = input_tensor.clone().detach() + torch.randn_like(input_tensor) * noise_std
        noisy_input.requires_grad_(True)

        # Forward and backward
        output = model(noisy_input.unsqueeze(0))  # batch dim
        
        output.backward()

        grad = noisy_input.grad.clone().detach()
        
        if multiply_by_input:
            grad = grad * noisy_input.detach()

        grads.append(grad)

    # Average all gradients
    return torch.stack(grads).mean(dim=0)

def grad_cam_ish(model, point):
    activations = {}
    gradients = {}
    def forward_hook(name):
        def hook(module, input, output):
            activations[name] = output.detach()
        return hook

    # Hook to capture gradients w.r.t. outputs
    def backward_hook(name):
        def hook(module, grad_input, grad_output):
            gradients[name] = grad_output[0].detach()
        return hook

    # Register hooks for each layer
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            module.register_forward_hook(forward_hook(name))
            module.register_backward_hook(backward_hook(name))

    # === Sample input ===
    input_point = torch.tensor(point, requires_grad=True, dtype=torch.float32)

    # Forward pass
    output = model(input_point)

    # Backward pass (ensure scalar output)
    output.backward()

    # Mean gradient for each neuron (shape: [32])
    weights = gradients['input_layer'].mean(dim=0)  # like alpha_k

    # Weighted sum of activations (shape: [1, 32])
    cam = (weights * activations['input_layer'].squeeze()).cpu().numpy()

    # Optional: ReLU to zero out negatives
    cam = np.maximum(cam, 0)

    input_contrib = cam @ model.input_layer.weight.detach().numpy()
    return input_contrib

def integrated_gradients(model, input_tensor, baseline=None, steps=50):
    """
    Compute Integrated Gradients for a single input.
    Args:
        model: PyTorch model
        input_tensor: 1D torch.Tensor, input point
        baseline: baseline input (same shape), default zero
        steps: number of interpolation steps

    Returns:
        IG attribution vector
    """
    model.eval()
    if baseline is None:
        baseline = torch.zeros_like(input_tensor)

    scaled_inputs = [
        baseline + (float(i) / steps) * (input_tensor - baseline)
        for i in range(1, steps + 1)
    ]

    grads = []
    for x in scaled_inputs:
        x = x.clone().detach().requires_grad_(True)
        output = model(x.unsqueeze(0))
        output.backward()
        grads.append(x.grad.detach())

    avg_grad = torch.stack(grads).mean(dim=0)
    ig = (input_tensor - baseline) * avg_grad
    return ig

from FeatureImportance.methods.FiniteDifferences import FiniteDifferences
from FeatureImportance.methods.LRP import TwoDWeights_LRPModel
from FeatureImportance.methods.LIME import LIME
from FeatureImportance.methods.SHAP import SHAP

lime = LIME(model, 1)
shap = SHAP(model)
fd = FiniteDifferences(model)
lrp = TwoDWeights_LRPModel(model)
model = model.to("cpu")
def get_LRP(model, points):
    lrp_exp = []
    POEs_torch = torch.tensor(points, dtype = torch.float32)
    for i in POEs_torch:
        lrp = TwoDWeights_LRPModel(model)
        _, lrp_expt = lrp.get_FI(i)
        
        lrp_exp.append(torch.stack(lrp_expt).numpy())
    lrp_exp = np.stack(lrp_exp)
    return lrp_exp

def get_grad(model, points):
    torch.set_grad_enabled(True)
    x = torch.tensor(points, dtype = torch.float32, requires_grad=True).to()
    model = model.requires_grad_(True)
    y = model(x)  # shape: [1, 3]
    
    jacobian = torch.zeros(points.shape[0], config["env"]["action_dim"], points.shape[1])

    for i in range(config["env"]["action_dim"]):
        # Create grad_outputs with 1 in i-th dimension, zeros elsewhere
        grad_outputs = torch.zeros_like(y)
        grad_outputs[:, i] = 1.0

        # Compute gradients
        grads = torch.autograd.grad(
            outputs=y,
            inputs=x,
            grad_outputs=grad_outputs,
            retain_graph=True,
            create_graph=True,
            allow_unused=True
        )[0]  # shape: [batch_size, input_dim]

    jacobian[:, i, :] = grads
    return jacobian

def get_SHAP(model, points):
    shap = SHAP(model)
    shap = shap.get_fi(points)[1]
    return shap

def get_LIME(model, points):
    lime = LIME(model, 2)
    _, lime = lime.get_fi(points)
    return lime

def get_SmoothGrad(model, points, multiply_by_input= False):
    points = torch.tensor(points, dtype = torch.float32)
    exps = []
    for i in points:
        exp = smooth_grad(model, i, multiply_by_input=multiply_by_input)
        exps.append(exp.detach().numpy())
    return np.stack(exps)

def get_GradCam(model, points):
    points = torch.tensor(points, dtype = torch.float32)
    exps = []
    for i in points:
        exp = grad_cam_ish(model, i)
        exps.append(exp)
    return np.stack(exps)

def get_IntegratedGradients(model, points):
    points = torch.tensor(points, dtype = torch.float32)
    exps = []
    for i in points:
        exp = integrated_gradients(model, i)
        exps.append(exp.detach().numpy())
    return np.stack(exps)

In [ ]:
POEs = np.array([[4.0, -4.0, -4.0,-4.0],[1.0,1.0,1.0,1.0],[0.0,0.0,0.0,0.0]])
# print("FD: ", get_grad(model, POEs))
# print("LRP: ", get_LRP(model, POEs))
# shapthing = get_SHAP(model, POEs)
# print("LIME: ", get_LIME(model, POEs))
# print("smooth_grad: ", get_SmoothGrad(model, POEs))
# print("grad_cam: ", get_GradCam(model, POEs))
# print("integrated_grads: ", get_GradCam(model, POEs))


RuntimeError: grad can be implicitly created only for scalar outputs